In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline

In [3]:
!unzip "/content/Data_Purdue_Team.zip" -d "/content/Data_Purdue_Team"

unzip:  cannot find or open /content/Data_Purdue_Team.zip, /content/Data_Purdue_Team.zip.zip or /content/Data_Purdue_Team.zip.ZIP.


In [4]:
read_file = pd.read_excel('/content/Data_Purdue_Team/Data/chat_messages_cleaned_repaired.xlsx')
read_file.to_csv(r'/content/Data_Purdue_Team/Data/chat_messages_cleaned.csv', index = None, header=True)
df = pd.read_csv('/content/Data_Purdue_Team/Data/chat_messages_cleaned.csv')
df.head()

,Unnamed: 0,participant__session__code,participant__session_id,participant__id_in_session,participant__code,channel,sender_role,body,timestamp,recipient__code,recipient_role,catalog,parameter_db
0,0,umtzl2qt,1,13,u76uc0tf,1-engine_design-3-1-3,crankshaft,Hi piston! My crankshaft depends on your bore ...,1554813515,r8fheelh,piston,0,0
1,1,umtzl2qt,1,11,r8fheelh,1-engine_design-3-1-3,piston,thank you!,1554813548,u76uc0tf,crankshaft,0,0
2,2,umtzl2qt,1,10,cbb51ylv,1-engine_design-2-1-5,pistonpin,How low can you go on the piston bore diameter?,1554813562,2wcqr3wm,piston,0,0
3,3,umtzl2qt,1,7,4ttkk1u5,1-engine_design-2-2-3,flywheel,"all my fos values are in the hundreds, i suspe...",1554813564,ntprw190,crankshaft,0,0
4,4,umtzl2qt,1,13,u76uc0tf,1-engine_design-3-2-3,crankshaft,Hi flywheel! My crankshaft depends on your fly...,1554813575,jyel22ui,flywheel,0,0


In [5]:
df['body']

0       Hi piston! My crankshaft depends on your bore ...
1                                              thank you!
2         How low can you go on the piston bore diameter?
3       all my fos values are in the hundreds, i suspe...
4       Hi flywheel! My crankshaft depends on your fly...
                              ...                        
7685                                   ok no one anything
7686                                    increase it to 50
7687                    what about thickness of flywheel?
7688                                    its at 12, lowest
7689                            raise it a little like 20
Name: body, Length: 7690, dtype: object

## Data Cleaning and Preprocessing

####Spell Correction

In [6]:
!pip install pyspellchecker

In [7]:
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    """custom function to check the spelling of text"""
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

for i in range(len(df['body'])):     
  text = df['body'][i]
  correct_spellings(text)

#### Punctuation Removal

In [8]:
import string  
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df['body_wo_punct'] = df['body'].apply(lambda text: remove_punctuation(text))
df['body'].head()

0    Hi piston! My crankshaft depends on your bore ...
1                                           thank you!
2      How low can you go on the piston bore diameter?
3    all my fos values are in the hundreds, i suspe...
4    Hi flywheel! My crankshaft depends on your fly...
Name: body, dtype: object

In [9]:
df['body_wo_punct'].head()

0    Hi piston My crankshaft depends on your bore d...
1                                            thank you
2       How low can you go on the piston bore diameter
3    all my fos values are in the hundreds i suspec...
4    Hi flywheel My crankshaft depends on your flyw...
Name: body_wo_punct, dtype: object

#### Lower Casing

In [11]:
# Create a new column as not to interfere with the existing data
def lower_casing(text):
  return text.lower()

df['body_lower'] = df['body'].apply(lambda text: lower_casing(text))
df['body_lower'].head()

0    hi piston! my crankshaft depends on your bore ...
1                                           thank you!
2      how low can you go on the piston bore diameter?
3    all my fos values are in the hundreds, i suspe...
4    hi flywheel! my crankshaft depends on your fly...
Name: body_lower, dtype: object

####StopWords Removal

In [12]:
!pip install stopwords

In [25]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [28]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["body_wo_stop"] = df["body_wo_punct"].apply(lambda text: remove_stopwords(text))
df["body_wo_stop"].head()

0    Hi piston My crankshaft depends bore diameter ...
1                                                thank
2                      How low go piston bore diameter
3            fos values hundreds suspect depends lot c
4    Hi flywheel My crankshaft depends flywheel sha...
Name: body_wo_stop, dtype: object

####Isolate Frequent Words

In [29]:
# Count of Frequent Words
from collections import Counter
cnt = Counter()
for text in df["body_wo_stop"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('mass', 1188),
 ('I', 1164),
 ('FOS', 739),
 ('fos', 626),
 ('diameter', 595),
 ('piston', 449),
 ('good', 438),
 ('need', 378),
 ('change', 378),
 ('think', 291)]

In [30]:
# Pick out Frequent Words 
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    """custom function to isolate the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

df["body_wo_stopfreq"] = df["body_wo_stop"].apply(lambda text: remove_freqwords(text))
df["body_wo_stopfreq"].head()

0    Hi My crankshaft depends bore dont yet know pr...
1                                                thank
2                                      How low go bore
3                values hundreds suspect depends lot c
4    Hi flywheel My crankshaft depends flywheel sha...
Name: body_wo_stopfreq, dtype: object

####Isolate Rare Words

In [31]:
n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    """custom function to isolate the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

df["body_wo_stopfreqrare"] = df["body_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
df["body_wo_stopfreqrare"] .head()

0    Hi My crankshaft depends bore dont yet know pr...
1                                                thank
2                                      How low go bore
3                values hundreds suspect depends lot c
4    Hi flywheel My crankshaft depends flywheel sha...
Name: body_wo_stopfreqrare, dtype: object

####Lemmatisation to bring out the root form 

In [32]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [34]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}

In [35]:
def lemmatize_words(text):
  """custom function to lemmatise words"""
  pos_tagged_text = nltk.pos_tag(text.split())
  return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

df["body_lemmatized"] = df["body"].apply(lambda text: lemmatize_words(text))
df["body_lemmatized"] .head()

0    Hi piston! My crankshaft depend on your bore d...
1                                           thank you!
2      How low can you go on the piston bore diameter?
3    all my fo value be in the hundreds, i suspect ...
4    Hi flywheel! My crankshaft depend on your flyw...
Name: body_lemmatized, dtype: object

######Note: The text don't have any HTML tags or emoticons. However, there are some chat words used which need to be converted into their full form.

####Chat Word Conversion

In [17]:
import csv
import re

In [18]:
# Testing function for slang.txt
def translator(user_string):
    user_string = user_string.split(" ")
    j = 0
    for _str in user_string:
        # File path which consists of Abbreviations.
        fileName = "/content/Data_Purdue_Team/Data/slang.txt"
        # File Access mode [Read Mode]
        accessMode = "r"
        with open(fileName, accessMode) as myCSVfile:
            # Reading file as CSV with delimiter as "=", so that abbreviation are stored in row[0] and phrases in row[1]
            dataFromFile = csv.reader(myCSVfile, delimiter="=")
            # Removing Special Characters.
            _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
            for row in dataFromFile:
                # Check if selected word matches short forms[LHS] in text file.
                if _str.upper() == row[0]:
                    # If match found replace it with its Abbreviation in text file.
                    user_string[j] = row[1]
            myCSVfile.close()
        j = j + 1
    # Replacing commas with spaces for final output.
    print(' '.join(user_string))

In [19]:
#translator('Hey Bhavika brb!!')

In [20]:
chat_words_str = pd.read_csv('/content/Data_Purdue_Team/Data/slang.txt', delimiter = "\t")

In [21]:
chat_words_str.T.reset_index().T
chat_words_str.columns = ['Abbrevations']
chat_words_str[['Abbrevations','Chat Words Converted']] = chat_words_str.Abbrevations.str.split("=",expand=True) 
chat_words_str

,Abbrevations,Chat Words Converted
0,AFK,Away From Keyboard
1,ASAP,As Soon As Possible
2,ATK,At The Keyboard
3,ATM,At The Moment
4,A3,"Anytime, Anywhere, Anyplace"
...,...,...
63,WTG,Way To Go
64,WUF,Where Are You From
65,W8,Wait...
66,7K,Sick


In [22]:
def chat_words_conversion(text):
  """custom function for chat words conversion"""
  new_text = []
  for w in text.split():
    if w.upper() in chat_words_str['Abbrevations']:
      new_text.append(chat_words_map_dict[w.upper()])
    else:
      new_text.append(w)
  return " ".join(new_text)

df["body_full_text"] = df["body"].apply(lambda text: chat_words_conversion(text))
df["body_full_text"] .head()

0    Hi piston! My crankshaft depends on your bore ...
1                                           thank you!
2      How low can you go on the piston bore diameter?
3    all my fos values are in the hundreds, i suspe...
4    Hi flywheel! My crankshaft depends on your fly...
Name: body_full_text, dtype: object

In [24]:
df.columns

Index(['Unnamed: 0', 'participant__session__code', 'participant__session_id',
       'participant__id_in_session', 'participant__code', 'channel',
       'sender_role', 'body', 'timestamp', 'recipient__code', 'recipient_role',
       'catalog', 'parameter_db', 'body_wo_punct', 'body_lower',
       'body_full_text'],
      dtype='object')